# Валидация данных и оценка модели

## Валидация данных. Методы валидации


Будем решать задачу классификации: классифицировать воду на пригодную и не пригодную для питья на основе её химического состава.

In [3]:
#Импортируем необходимые модули

#Для матричных вычислений
import numpy as np 
#Для анализа и предобработки данных
import pandas as pd
#Для визуализации
import matplotlib.pyplot as plt 
import seaborn as sns

#Метрики
from sklearn import metrics
#Методы разделения и валидации
from sklearn import model_selection

#Линейные модели 
from sklearn import linear_model
#Деревья решений
from sklearn import tree
#Стиль отрисовки seaborn 
plt.style.use('seaborn-v0_8')
%matplotlib inline

In [4]:
#Загрузим наши данные
water_data = pd.read_csv('data/water_potability.csv')
display(water_data.head())

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0


In [6]:
#Выведем информацию о пропусках в данных в процентном соотношении
display(water_data.isnull().mean() * 100)

ph                 14.987790
Hardness            0.000000
Solids              0.000000
Chloramines         0.000000
Sulfate            23.840049
Conductivity        0.000000
Organic_carbon      0.000000
Trihalomethanes     4.945055
Turbidity           0.000000
Potability          0.000000
dtype: float64

In [7]:
#Заполняем пропуски
water_data['ph'] = water_data['ph'].fillna(water_data.groupby('Potability')
                                           ['ph'].transform('median'))
water_data['Sulfate'] = water_data['Sulfate'].fillna(water_data.groupby('Potability')
                                           ['Sulfate'].transform('median'))
water_data['Trihalomethanes'] = water_data['Trihalomethanes'].fillna(water_data.groupby('Potability')
                                           ['Trihalomethanes'].transform('median'))

In [8]:
#Убедимся в отсутствии пропусков
display(water_data.isnull().mean() * 100)

ph                 0.0
Hardness           0.0
Solids             0.0
Chloramines        0.0
Sulfate            0.0
Conductivity       0.0
Organic_carbon     0.0
Trihalomethanes    0.0
Turbidity          0.0
Potability         0.0
dtype: float64

In [9]:
#Разделим набор данных на матрицу наблюдений 
#и вектор правильных ответов
X = water_data.drop('Potability', axis=1)
y = water_data['Potability']

### Основные методы валидации данных

### HOLD-OUT
Его идея состоит в том, что для проверки модели мы просто случайным образом разбиваем весь набор данных на обучающую, валидационную и тестовую выборки (последняя — по желанию).


Реализация метода в sklearn

In [10]:
#Пример разбиения данных
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X, 
                                                                      y, 
                                                                      test_size=0.2,
                                                                      random_state=42)

In [11]:
#Проверим размеры полученных выборок
print('Train shape: {}'.format(X_train.shape))
print('Test shape: {}'.format(X_valid.shape))

Train shape: (2620, 9)
Test shape: (656, 9)
